# 4-Oscar Prediction with AutoML
After out dataframe has been assemlbed (see scraping and table_assembling) notebooks we have the data we need to make predictions on the Best Picture winner. [AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) represents a quick, but powerful route though the Machine Learning process. H2O's AutoML runs many models through the dataset and using cross-validation, picks the best one. For my purposes I use it to confirm/compare to the Preferential Balloting Random Forest model I created.
If you are gunning to win your office's Oscar pool, scroll down to see the results.

In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('./data/processed_results/osc_df')

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.371-b11, mixed mode)
  Starting server from C:\Users\Aleksandra Czaplak\AppData\Roaming\Python\Python38\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALEKSA~1\AppData\Local\Temp\tmplskjmudb
  JVM stdout: C:\Users\ALEKSA~1\AppData\Local\Temp\tmplskjmudb\h2o_Aleksandra_Czaplak_started_from_python.out
  JVM stderr: C:\Users\ALEKSA~1\AppData\Local\Temp\tmplskjmudb\h2o_Aleksandra_Czaplak_started_from_python.err


C:\Users\Aleksandra Czaplak\AppData\Roaming\Python\Python38\site-packages\h2o\backend\server.py:386: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_Aleksandra_Czaplak_4by8tr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,241.3 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


First Year of Existance. This data will be used below
- golden_globes 1943
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# H2O's Auto ML

In [5]:
# Auto ML uses Cross Validation, so we do not specifiy a validation set
train = full_table.loc[((full_table['year'] < 2022) & (full_table['year'] >= min_year))]

print('training set contains:', train.shape[0], 'movies')

training set contains: 186 movies


In [8]:
train = train.drop('Unnamed: 0', axis=1)
train.columns

Index(['year', 'film', 'wiki', 'winner', 'nominations', 'Oscar_win',
       'nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga',
       'winner_dga', 'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [9]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [10]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
        'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes','nominations']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()

# Run AutoML for 100 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=100, seed=1
                , keep_cross_validation_predictions= True
               , exclude_algos = ['StackedEnsemble'])

aml.train(x=x, y=y, training_frame=train1)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
14:55:10.19: AutoML: XGBoost is not available; skipping it.
14:55:10.166: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]


14:55:11.407: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]
14:55:11.408: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 186.0.
14:55:11.419: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

██
14:55:12.619: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

█
14:55:13.338: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

█
14:55:13.966: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]

██
14:55:14.603: _train param, Dropping bad and constant columns: [winner_cannes, nom_cannes]
14:55:15.207: _train

In [8]:
top_model = aml.leader
top_model

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20200130_221603_model_32


Model Summary: 


,,number_of_trees
0,,183.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.07864349634776278
RMSE: 0.28043447781569725
LogLoss: 0.2904010718951874
Mean Per-Class Error: 0.08749999999999991
AUC: 0.9583333333333334
AUCPR: 0.785851912395023
Gini: 0.9166666666666667

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3950708508491516: 


,,0,1,Error,Rate
0,0,130.0,5.0,0.037,(5.0/135.0)
1,1,6.0,18.0,0.25,(6.0/24.0)
2,Total,136.0,23.0,0.0692,(11.0/159.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.8875,0.09003038,0.78125,0.8125,0.9375,0.90625,1.0
1,auc,0.8724014,0.09534497,0.8055556,0.8064516,0.95,0.8,1.0
2,aucpr,0.2601506,0.29980978,0.7482639,0.071428575,0.33333334,0.14772727,0.0
3,err,0.1125,0.09003038,0.21875,0.1875,0.0625,0.09375,0.0
4,err_count,3.6,2.8809721,7.0,6.0,2.0,3.0,0.0
5,f0point5,0.5944418,0.3485882,0.88709676,0.1724138,0.5555556,0.35714287,1.0
6,f1,0.61505747,0.29621416,0.7586207,0.25,0.6666667,0.4,1.0
7,f2,0.68101496,0.2386768,0.6626506,0.45454547,0.8333333,0.45454547,1.0
8,lift_top_group,6.5555553,13.686525,1.7777778,0.0,0.0,0.0,31.0
9,logloss,0.36044943,0.23865479,0.77814555,0.31771538,0.21953076,0.28969845,0.19715707



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-01-30 22:17:02,36.485 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000,0.849057
1,,2020-01-30 22:17:02,36.492 sec,5.0,0.481111,0.656050,0.882562,0.647821,6.625,0.088050
2,,2020-01-30 22:17:02,36.496 sec,10.0,0.463173,0.621973,0.911883,0.710444,6.625,0.094340
3,,2020-01-30 22:17:02,36.501 sec,15.0,0.446822,0.591754,0.917438,0.744178,6.625,0.075472
4,,2020-01-30 22:17:02,36.506 sec,20.0,0.431995,0.564939,0.912809,0.727756,6.625,0.081761
5,,2020-01-30 22:17:02,36.511 sec,25.0,0.417822,0.539771,0.913735,0.735334,6.625,0.081761
6,,2020-01-30 22:17:02,36.516 sec,30.0,0.405790,0.518654,0.919599,0.752560,6.625,0.081761
7,,2020-01-30 22:17:02,36.522 sec,35.0,0.394182,0.498472,0.919290,0.750756,6.625,0.081761
8,,2020-01-30 22:17:02,36.528 sec,40.0,0.383642,0.480286,0.920525,0.757818,6.625,0.075472
9,,2020-01-30 22:17:02,36.534 sec,45.0,0.374217,0.464044,0.920525,0.757818,6.625,0.075472



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,winner_pga,151.066833,1.000000,0.445544
1,winner_dga,114.597305,0.758587,0.337983
2,winner_sag,29.370192,0.194419,0.086622
3,Nominations,19.300804,0.127763,0.056924
4,year,8.320313,0.055077,0.024539
5,nom_sag,6.095071,0.040347,0.017976
6,winner_bafta,4.053117,0.026830,0.011954
7,winner_gg_drama,2.857199,0.018913,0.008427
8,nom_bafta,2.735089,0.018105,0.008067
9,nom_dga,0.468218,0.003099,0.001381


## Predict the winner

In [9]:
# Predict on 2019's films
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
preds = top_model.predict(test)

preds

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.872341,0.127659
0,0.871212,0.128788
0,0.871916,0.128084
0,0.871639,0.128361
0,0.872341,0.127659
0,0.872341,0.127659
1,0.508026,0.491974
0,0.871212,0.128788
0,0.805078,0.194922


In [11]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [12]:
final_rankings = test_pd[['film','probA']].sort_values('probA', ascending = False)
final_rankings['%_confidence'] = final_rankings['probA']/final_rankings['probA'].sum() * 100
final_rankings

,film,probA,%_confidence
6,1917 (2019 film),0.491974,31.061029
8,Parasite (2019 film),0.194922,12.306475
1,The Irishman,0.128788,8.131125
7,Once Upon a Time in Hollywood,0.128788,8.131125
3,Joker (2019 film),0.128361,8.104137
2,Jojo Rabbit,0.128084,8.086673
0,Ford v Ferrari,0.127659,8.059812
4,Little Women (2019 film),0.127659,8.059812
5,Marriage Story,0.127659,8.059812


# And the Oscar goes to...

In [13]:
bp_winner = np.array(final_rankings.reset_index())[0][1].split('(')[0].strip()
print(f'And the Oscar goes to...\n🎉🏆{bp_winner}🏆🎉')

And the Oscar goes to...
🎉🏆1917🏆🎉
